In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme = 'onedork', grid = False, ticks = True)

In [ ]:
import numpy as np
import pandas as pd
# ML
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

import cv2
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
from torchsummary import summary

# Plots
import matplotlib.pyplot as plt

# Utils
from collections import OrderedDict
from tqdm import tqdm, trange
import os

## Utils

# Data

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
# The ImageFolder dataset
transform = transforms.Compose([transforms.ToTensor()])
dataset = torchvision.datasets.ImageFolder('../datasets/money/', transform=transform)

In [ ]:
dataset

In [ ]:
dataset.classes

In [ ]:
dataset.samples[:5]

In [ ]:
sample, label =  next(iter(dataset))

In [ ]:
plt.imshow(sample.T) # Channels first

In [ ]:
# We can use our dataset with the dataloader! 
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True)
samples, labels = next(iter(dataloader))

In [ ]:
plt.imshow(samples[0].T)
plt.show()

In [ ]:
from torch.utils.data import random_split
train_len = int(.8 * len(dataset))
test_len = len(dataset) - train_len
train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

# Models and layers

## Convolutions

Videos 
- https://www.youtube.com/watch?v=YRhxdVk_sIs - Deeplizard
- https://www.youtube.com/watch?v=x_VrgWTKkiM - Tensorflow
- https://www.youtube.com/watch?v=py5byOOHZM8 - Mike Pound <3 
- https://www.youtube.com/watch?v=pj9-rr1wDhM - Visualization
- https://www.youtube.com/watch?v=f0t-OCG79-U - Viz 2

Play with it 
- https://www.cs.ryerson.ca/~aharley/vis/



Convolution output dimension
$$W_{out} = \dfrac {W_{in} - K + 2P} S + 1$$

Where 
- $W_{in}$ = Input size
- $K$ = Filter size
- $S$ = Stride
- $P$ = Padding

In [ ]:
#?nn.Conv2d

In [ ]:
conv = nn.Conv2d(3, 32, kernel_size = 5, stride = 2, padding = 2, padding_mode = 'zeros')

In [ ]:
conv_output = conv(sample.unsqueeze(0))

In [ ]:
# out_dim  = (128 - 5 + 4) / 1 + 1 = 128
conv_output.shape

## Max pool layer

In [ ]:
#nn.MaxPool2d

In [ ]:
maxpool = nn.MaxPool2d(kernel_size = 2, stride = 2)

In [ ]:
maxpool_output = maxpool(conv_output)
maxpool_output.shape

## Model

In [ ]:
class CNNModel(nn.Module):
    def __init__(self,output_dim):
        super(CNNModel, self).__init__()
        
        # First group
        self.conv1 = nn.Conv2d(3, 8, kernel_size = 5, stride = 2, padding = 2, padding_mode = 'zeros')
        # out_dim = 64 x 64
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        # out_dim = 32 x 32
        
        # Second group
        self.conv2 = nn.Conv2d(8, 16, kernel_size = 5, stride = 2, padding = 2, padding_mode = 'zeros')
        # out_dim = 16 x 16
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        # out_dim = 8 x 8 

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(8 * 8 * 16, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, output_dim)
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return F.softmax(x, dim = 1)

# Training loop

In [ ]:
input_shape = tuple(next(iter(dataset))[0].shape)
output_shape = len(dataset.classes)
model = CNNModel(output_shape)
#model.cuda()

In [ ]:
summary(model = model, input_size=(input_shape), batch_size = 32, device = 'cpu')

In [ ]:
for p in model.parameters():
    print(p.shape)
    #print(p)

In [ ]:
model.cuda()

In [ ]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

In [ ]:
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
epochs = 15
batch_size = 64
accs = []
losses = []
val_losses = []
val_accs = []
for epoch in (t:= trange(epochs)):
    # Get batches
    model.train()
    loss = 0.
    acc = 0.
    num_batches = 0
    for X_batch, y_batch in trainloader:
        num_batches+=1
        X_batch = X_batch.cuda()
        y_batch = y_batch.cuda()
        y_pred = model(X_batch)
        
        loss_batch = loss_function(y_pred, y_batch)    
        loss += loss_batch.item()
        acc += accuracy_score(torch.argmax(y_pred.cpu(), axis = 1), y_batch.cpu())
        
        optimizer.zero_grad() # don't forget this
        loss_batch.backward()
        optimizer.step()
    
    acc /= num_batches
    loss /= num_batches
    losses.append(loss)
    accs.append(acc)
    
    # Validation set
    model.eval()
    num_batches = 0
    val_acc = 0.
    val_loss = 0.
    for X_batch, y_batch in testloader:
        num_batches+=1
        X_batch = X_batch.cuda()
        y_batch = y_batch.cuda()
        
        y_pred = model(X_batch)
        val_acc += accuracy_score(torch.argmax(y_pred.cpu(), axis = 1), y_batch.cpu())
        loss_batch = loss_function(y_pred, y_batch)    
        val_loss += loss_batch.item()
        
    val_acc /= num_batches
    val_loss /= num_batches
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    t.set_description(f"(Loss, Acc)--Train : {round(loss, 2), round(acc, 2)}, Test : {round(val_loss, 2), round(val_acc, 2)}")
    

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (20, 5))

axs[0].set_ylim(0, 1.1)
axs[0].plot(losses)
axs[0].plot(val_losses)

axs[1].set_ylim(0, 1.1)
axs[1].plot(accs)
axs[1].plot(val_accs)


# Let's look at the convolutions

In [ ]:
dataloader = DataLoader(dataset, batch_size = 32, shuffle = False)
samples, labels = next(iter(dataloader))


In [ ]:
sample = samples[1]
plt.imshow(sample.T)

In [ ]:
conv = nn.Conv2d(3, 8, kernel_size = 5, stride = 2, padding = 2, padding_mode = 'zeros')

conv_out_untrained = conv(sample.unsqueeze(0)).detach().numpy().squeeze(0)

conv_out_untrained.shape

In [ ]:
plt.figure(figsize = (10, 10))
plt.imshow(np.concatenate(conv_out_untrained.reshape(4, 64 * 2, 64), axis = 1), cmap='Blues')

In [ ]:
conv_out_trained = model.conv1(sample.unsqueeze(0).cuda()).cpu().detach().numpy().squeeze(0)

conv_out_trained.shape

In [ ]:
plt.figure(figsize = (10, 10))
plt.imshow(np.concatenate(conv_out_trained.reshape(4, 64 * 2, 64), axis = 1), cmap='Blues')

In [ ]:
model.state_dict().keys()

# Transfer learning

In [ ]:
# https://github.com/pytorch/vision/issues/616


In [ ]:
resnet18 = torchvision.models.resnet18(pretrained = True) # Takes multiple of 32 as input 

In [ ]:
summary(resnet18, input_size=(3, 128, 128), batch_size = 32, device = "cpu")

In [ ]:
fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512,128)),
    ('dropout', nn.Dropout(p = .5)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(128,2)),
    ('output', nn.LogSoftmax(dim=1))
]))
resnet18.fc = fc

In [ ]:
summary(resnet18, input_size=(3, 128, 128), batch_size = 32, device = "cpu")

In [ ]:
# Freeze the parameters 
for param in model.parameters():
    param.requires_grad = False 
# Unfreeze last layer
for param in resnet18.fc.parameters():
    param.requires_grad = True
    

### Training loop

In [ ]:
resnet18 = resnet18.cuda()

In [ ]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(resnet18.parameters(), lr=learning_rate) # change resnet here
loss_function = nn.CrossEntropyLoss()

batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
epochs = 10
accs = []
losses = []
resnet18.train() # change resnet here
for epoch in (t:= trange(epochs)):
    # Get batches
    loss = 0.
    acc = 0.
    num_batches = 0
    for X_batch, y_batch in trainloader:
        num_batches+=1
        X_batch = X_batch.cuda()
        y_batch = y_batch.cuda()
        y_pred = resnet18(X_batch)
        
        loss_batch = loss_function(y_pred, y_batch)    
        loss += loss_batch.item()
        acc += accuracy_score(torch.argmax(y_pred.cpu(), axis = 1), y_batch.cpu())
        
        optimizer.zero_grad() # don't forget this
        loss_batch.backward()
        optimizer.step()
    
    acc /= num_batches
    loss /= num_batches
    losses.append(loss)
    accs.append(acc)

    # Validation set
    model.eval()
    num_batches = 0
    val_acc = 0.
    val_loss = 0.
    for X_batch, y_batch in testloader:
        num_batches+=1
        X_batch = X_batch.cuda()
        y_batch = y_batch.cuda()
        
        y_pred = model(X_batch)
        val_acc += accuracy_score(torch.argmax(y_pred.cpu(), axis = 1), y_batch.cpu())
        loss_batch = loss_function(y_pred, y_batch)    
        val_loss += loss_batch.item()
        
    val_acc /= num_batches
    val_loss /= num_batches
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    t.set_description(f"(Loss, Acc) -- Train : {round(loss, 2), round(acc, 2)}, Test : {round(val_loss, 2), round(val_acc, 2)}")
    

    

In [ ]:
del X_batch, y_batch
torch.cuda.empty_cache()

In [ ]:
plt.figure(figsize = (10, 7))
plt.ylim(0, 1.1)
plt.legend(['Loss','Accuracy'])
plt.plot(losses)
plt.plot(accs)

# Save and load models

In [ ]:
torch.save(resnet18.state_dict(), "models/my_models/my_resnet18.pth")

In [ ]:
loaded_resnet18 =  torchvision.models.resnet18(pretrained = False) # Takes multiple of 32 as input 
fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512,128)),
    ('dropout', nn.Dropout(p = .5)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(128,2)),
    ('output', nn.LogSoftmax(dim=1))
]))
loaded_resnet18.fc = fc

In [ ]:
loaded_resnet18.load_state_dict(torch.load("models/my_models/my_resnet18.pth"))
loaded_resnet18.eval()